In [5]:
import json
import random
import pandas as pd
import pymysql
from faker import Faker
from datetime import datetime, timedelta,time 
from dateutil.relativedelta import relativedelta  # Import relativedelta for date calculations
from dateutil.parser import parse  # Import parse for date string to datetime conversion
import pymysql
from datetime import datetime, timezone
from datetime import datetime
import pytz
import hashlib
import re
pst = pytz.timezone('America/Los_Angeles')
pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [14]:
# Establish a connection to the database
connection = pymysql.connect(
    host='34.94.165.202',
    user='root',  # enter your username
    password='thereisnopassword',  # enter your password
    database='banking_system'
)

In [16]:

# Establish a connection to the database
connection = pymysql.connect(
    host='34.94.165.202',
    user='root',  # enter your username
    password='thereisnopassword',  # enter your password
    database='banking_system'
)
def hash_password(password):
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    return hashed_password

def signup(connection):
    print("***************************************************************")
    print("Welcome to the Signup Process")

    # Get employee code
    employee_code = input("Enter your Employee Code: ")

    # Decode employee code into employee ID and branch ID
    try:
        employee_id, branch_id = employee_code[:-1], employee_code[-1]

        with connection.cursor() as cursor:
            # Check if the employee exists and has the position 'Teller'
            cursor.execute("SELECT * FROM Employee WHERE EmployeeID = %s AND BranchID = %s", (employee_id, branch_id))
            employee = cursor.fetchone()

            if employee and employee[1] == 'Teller':
                while True:
                    try:
                        # Get user details
                        name = input("Enter the new user's full name: ")
                        dob = input("Enter the new user's date of birth (YYYY-MM-DD): ")
                        email = input("Enter the new user's email: ").lower()

                        # Check if the email already exists
                        cursor.execute("SELECT * FROM Person WHERE Email = %s", email)
                        existing_user = cursor.fetchone()

                        if existing_user:
                            print("Email already exists. Please choose a different email.")
                            continue  # Restart the loop to get new user details

                        phone_number = input("Enter the new user's phone number: ")
                        address = input("Enter the new user's address: ")
                        password = input("Enter the new user's password: ")

                        # Hash the password
                        hashed_password = hash_password(password)

                        # Display user information for confirmation
                        user_info_table = pd.DataFrame(
                            {
                                'Name': [name],
                                'Date of Birth': [dob],
                                'Email': [email],
                                'Phone Number': [phone_number],
                                'Address': [address],
                            }
                        )
                        print("Please review the entered information:")
                        print(user_info_table)

                        # Ask user to confirm or edit information
                        confirm_edit = input("Do you want to confirm this information? (yes/no): ").lower()

                        if confirm_edit == 'no':
                            # Ask which information they want to edit
                            edit_field = input("Which information would you like to edit? "
                                               "(name/date of birth/email/phone number/address/password): ").lower()

                            # Update the corresponding field
                            if edit_field == 'name':
                                name = input("Enter the corrected full name: ")
                            elif edit_field == 'date of birth':
                                dob = input("Enter the corrected date of birth (YYYY-MM-DD): ")
                            elif edit_field == 'email':
                                email = input("Enter the corrected email: ").lower()
                                # Check if the corrected email already exists
                                cursor.execute("SELECT * FROM Person WHERE Email = %s", email)
                                existing_user = cursor.fetchone()
                                if existing_user:
                                    print("Email already exists. Update canceled.")
                                    continue  # Restart the loop to get new user details
                            elif edit_field == 'phone number':
                                phone_number = input("Enter the corrected phone number: ")
                            elif edit_field == 'address':
                                address = input("Enter the corrected address: ")
                            elif edit_field == 'password':
                                password = input("Enter the corrected password: ")
                                hashed_password = hash_password(password)

                        # Insert user details into the database
                        cursor.execute("INSERT INTO Person (`Name`, DOB, Email, PhoneNumber, Address) VALUES (%s, %s, %s, %s, %s)",
                                       (name, dob, email, phone_number, address))
                        person_id = cursor.lastrowid

                        cursor.execute("INSERT INTO Employee (Position, PersonID, BranchID) VALUES (%s, %s, %s)", ('Customer', person_id, branch_id))

                        cursor.execute("INSERT INTO UserCredentials (PersonID, Email, Password) VALUES (%s, %s, %s)",
                                       (person_id, email, hashed_password))

                        connection.commit()
                        print("Signup successful!")

                        # Ask if the user wants to sign up another user
                        signup_another = input("Do you want to sign up another user? (yes/no): ").lower()
                        if signup_another != 'yes':
                            break

                    except Exception as e:
                        connection.rollback()
                        print("Signup error:", e)
                        break  # Exit the loop in case of an error

            else:
                print("Invalid employee code or position. Only Tellers are allowed to sign up new users.")

    except Exception as e:
        connection.rollback()
        print("Unexpected error:", e)

# Call the signup function
signup(connection)


***************************************************************
Welcome to the Signup Process
Enter your Employee Code: 203
Enter the new user's full name: sd
Enter the new user's date of birth (YYYY-MM-DD): 1993-12-12
Enter the new user's email: sd@gmail.com
Enter the new user's phone number: 1-223-233-2333
Enter the new user's address: 34 Frive San Jose
Enter the new user's password: qwer
Please review the entered information:
  Name Date of Birth         Email    Phone Number            Address
0   sd    1993-12-12  sd@gmail.com  1-223-233-2333  34 Frive San Jose
Do you want to confirm this information? (yes/no): yes
Signup successful!
Do you want to sign up another user? (yes/no): no
SignUp Successfully


In [21]:
print("***************************************************************")
print("Welcome to the Signup Process")
class BankSignup:
    def __init__(self, host, user, password, database):
        self.connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )

    def hash_password(self, password):
        hashed_password = hashlib.sha256(password.encode()).hexdigest()
        return hashed_password

    def signup(self):
        

        # Get employee code
        employee_code = input("Enter your Employee Code: ")

        try:
            employee_id, branch_id = employee_code[:-1], employee_code[-1]

            with self.connection.cursor() as cursor:
                cursor.execute("SELECT * FROM Employee WHERE EmployeeID = %s AND BranchID = %s", (employee_id, branch_id))
                employee = cursor.fetchone()

                if employee and employee[1] == 'Teller':
                    while True:
                        try:
                            # Get user details
                            name = input("Enter the new user's full name: ")
                            dob = input("Enter the new user's date of birth (YYYY-MM-DD): ")
                            email = input("Enter the new user's email: ").lower()

                            cursor.execute("SELECT * FROM Person WHERE Email = %s", email)
                            existing_user = cursor.fetchone()

                            if existing_user:
                                print("Email already exists. Please choose a different email.")
                                continue

                            phone_number = input("Enter the new user's phone number: ")
                            address = input("Enter the new user's address: ")
                            password = input("Enter the new user's password: ")

                            hashed_password = self.hash_password(password)

                            user_info_table = pd.DataFrame(
                                {
                                    'Name': [name],
                                    'Date of Birth': [dob],
                                    'Email': [email],
                                    'Phone Number': [phone_number],
                                    'Address': [address],
                                }
                            )
                            print("Please review the entered information:")
                            print(user_info_table)

                            confirm_edit = input("Do you want to confirm this information? (yes/no): ").lower()

                            if confirm_edit == 'no':
                                edit_field = input("Which information would you like to edit? "
                                                   "(name/date of birth/email/phone number/address/password): ").lower()

                                if edit_field == 'name':
                                    name = input("Enter the corrected full name: ")
                                elif edit_field == 'date of birth':
                                    dob = input("Enter the corrected date of birth (YYYY-MM-DD): ")
                                elif edit_field == 'email':
                                    email = input("Enter the corrected email: ").lower()
                                    cursor.execute("SELECT * FROM Person WHERE Email = %s", email)
                                    existing_user = cursor.fetchone()
                                    if existing_user:
                                        print("Email already exists. Update canceled.")
                                        continue
                                elif edit_field == 'phone number':
                                    phone_number = input("Enter the corrected phone number: ")
                                elif edit_field == 'address':
                                    address = input("Enter the corrected address: ")
                                elif edit_field == 'password':
                                    password = input("Enter the corrected password: ")
                                    hashed_password = self.hash_password(password)

                            cursor.execute("INSERT INTO Person (`Name`, DOB, Email, PhoneNumber, Address) VALUES (%s, %s, %s, %s, %s)",
                                           (name, dob, email, phone_number, address))
                            person_id = cursor.lastrowid

                            cursor.execute("INSERT INTO UserCredentials (PersonID, Email, Password) VALUES (%s, %s, %s)",
                                           (person_id, email, hashed_password))

                            self.connection.commit()
                            print("Signup successful!")

                            signup_another = input("Do you want to sign up another user? (yes/no): ").lower()
                            if signup_another != 'yes':
                                break

                        except Exception as e:
                            self.connection.rollback()
                            print("Signup error:", e)
                            break

                else:
                    print("Invalid employee code or position. Only Tellers are allowed to sign up new users.")

        except Exception as e:
            self.connection.rollback()
            print("Unexpected error:", e)

# Example usage:
bank_signup = BankSignup(
    host='34.94.165.202',
    user='root',
    password='thereisnopassword',
    database='banking_system'
)

bank_signup.signup()


***************************************************************
Welcome to the Signup Process
Enter your Employee Code: 41
Enter the new user's full name: Hoa Nguyen
Enter the new user's date of birth (YYYY-MM-DD): 1993-12-25
Enter the new user's email: hoa@gmail.com
Enter the new user's phone number: 1-234-234-2345
Enter the new user's address: 12 San Jose 
Enter the new user's password: hoa
Please review the entered information:
         Name Date of Birth          Email    Phone Number       Address
0  Hoa Nguyen    1993-12-25  hoa@gmail.com  1-234-234-2345  12 San Jose 
Do you want to confirm this information? (yes/no): yes
Signup successful!
Do you want to sign up another user? (yes/no): no
